In [1]:
import pandas as pd
import numpy as np
import requests
from urllib.parse import urlencode

In [2]:
assessments_url = 'https://disk.yandex.ru/d/WG2u0PPEfUCWvw'
courses_url = 'https://disk.yandex.ru/d/51xrZHoHNy3R3Q'
studentAssessment_url= 'https://disk.yandex.ru/d/eCCGfl52Xp_4Lw'
studentRegistration_url = 'https://disk.yandex.ru/d/H_qZLNN9wgnAwQ'

In [3]:
def ydisk_api(public_key: str) -> str:
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(url)
    download = response.json()['href']
    return download

In [4]:
assessments = pd.read_csv(ydisk_api(assessments_url))
courses = pd.read_csv(ydisk_api(courses_url))
studentAssessment = pd.read_csv(ydisk_api(studentAssessment_url))
studentRegistration = pd.read_csv(ydisk_api(studentRegistration_url))

### Кейс 1
Сколько студентов успешно сдали только один курс?

In [9]:
df_full = assessments.merge(studentAssessment).merge(courses)

In [10]:
df_full['date_submitted'].nlargest?

Object `nlargest` not found.


In [ ]:
df_full['date_submitted'].nlargest

In [11]:
len(df_full['date_submitted'])

173912

In [12]:
df = df_full.query('assessment_type == "Exam" & score >= 40') \
    .groupby('id_student', as_index=False) \
    .agg({'code_module': "count"})

In [13]:
df[df['code_module']==1]

,id_student,code_module
0,23698,1
1,24213,1
2,27116,1
3,28046,1
4,28787,1
...,...,...
4092,2694886,1
4093,2694933,1
4094,2695608,1
4095,2697181,1


In [14]:
print(f'{df[df["code_module"]==1].count()[0]} студентов успешно сдали только один курс')

3802 студентов успешно сдали только один курс


### Кейс 2
Найдите и отсортируйте id экзаменов в рамках курса по возрастанию уровня завершаемости:

In [15]:
df2 = df_full.query('assessment_type == "Exam"')

In [16]:
df2['pass'] = df2['score'].apply(lambda x: x >=40)

/Users/kate/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
df2.groupby('id_assessment', as_index=False) \
   .agg(success=('pass', 'sum'), total_attempts=('pass', 'count')) \
   .assign(rate=lambda x: x.success / x.total_attempts * 100) \
   .sort_values(by='rate')

,id_assessment,success,total_attempts,rate
2,25340,504,602,83.720930
1,24299,1019,1168,87.243151
5,25368,842,950,88.631579
0,24290,664,747,88.888889
3,25354,878,968,90.702479
4,25361,485,524,92.557252


id экзаменов в рамках курса по возрастанию уровня завершаемости [25340, 24299, 25368, 24290, 25354, 25361]

### Кейс 3
Выявите самые популярные предметы (ТОП-3) по количеству регистраций на них:

In [18]:
studentRegistration \
                    .groupby('code_module') \
                    .agg({'id_student': "nunique"}) \
                    .sort_values(by='id_student', ascending=False) \
                    .head(3)

,id_student
code_module,
BBB,7692
FFF,7397
DDD,5848


### Кейс 4
Выявите предметы с самым большим оттоком* (ТОП-3):  
*Отток в данном случае — это отмена регистрации на предмет.

In [19]:
studentRegistration \
            .groupby('code_module') \
            .agg({'date_unregistration': "count"}) \
            .sort_values(by='date_unregistration', ascending=False) \
            .head(3)

,date_unregistration
code_module,
FFF,2380
BBB,2377
DDD,2235


### Кейс 5
Используя pandas, в период с начала 2013 по конец 2014 выявите семестр с самой низкой завершаемостью курсов:

In [20]:
df2.groupby('code_presentation', as_index=False) \
   .agg(success=('pass', 'sum'), total_attempts=('pass', 'count')) \
   .assign(rate=lambda x: x.success / x.total_attempts * 100) \
   .sort_values(by='rate')

,code_presentation,success,total_attempts,rate
0,2013B,504,602,83.720930
3,2014J,1861,2118,87.865911
2,2014B,1149,1271,90.401259
1,2013J,878,968,90.702479


Семестр с самой низкой завершаемостью курсов - 2013B

### Кейс 6
Используя pandas, в период с начала 2013 по конец 2014 выявите семестр с самыми долгими средними сроками сдачи курсов:

In [21]:
df_full.query('assessment_type == "Exam"') \
            .groupby('code_presentation') \
            .agg({"date_submitted": "mean"}) \
            .sort_values(by='date_submitted', ascending=False)

,date_submitted
code_presentation,
2014J,243.674221
2013J,239.505165
2014B,232.988985
2013B,230.179402


Семестр с самыми долгими средними сроками сдачи курсов: 2014J

### Кейс 7
Часто для качественного анализа аудитории используют подходы, основанные на сегментации. Используя python, постройте адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. В адаптированной кластеризации можете выбрать следующие метрики:

R — среднее время сдачи одного экзамена,

F — завершаемость курсов, 

M — среднее количество баллов, получаемое за экзамен.

Для каждого RFM-сегмента постройте границы метрик recency, frequency и monetary для интерпретации этих кластеров.


Чему равна минимальная граница по recency?



In [22]:
#создаем df с колонками recency, monetary, агрегируя по дате-среднее и оценке-среднее
rm = df_full.query("assessment_type == 'Exam'") \
            .groupby("id_student", as_index=False) \
            .agg(recency=('date_submitted', 'mean'), monetary=("score", "mean"))

In [23]:
#rm.describe() #229

### Кейс 8
Чему равна максимальная граница по recency?

In [24]:
#rm.describe() #285

### Кейс 9
Чему равна минимальная граница по monetary?

In [25]:
#rm.describe() #0

### Кейс 10
Чему равна максимальная граница по monetary?

In [26]:
#rm.describe() #100

### Кейс 11
Используя логику:

1) по recency (среднее время сдачи одного экзамена) можно получить 2, если значение recency меньше либо равно медиане по recency. В остальных случаях — 1.

2) по frequency можно получить 1, если значение frequency меньше 50. Можно получить 2, если значение по frequency меньше 100. Можно получить 3 в остальных случаях.

3) по monetary можно получить 1, если значение monetary меньше 40. Можно получить 2, если значение по monetary меньше либо равно 80. Можно получить 3 в остальных случаях.

Сколько клиентов попадут в кластер 232?

1. создаем дф где считаем сколько экзаменов успешно сдал каждый студент

In [27]:
success_pass = df_full.query("assessment_type == 'Exam' & score >= 40") \
              .groupby("id_student", as_index=False) \
              .agg(success=("id_assessment", "count"))

2. считаем сколько в принципе сдавал каждый студент:

In [28]:
failure = df_full.query("assessment_type == 'Exam'") \
         .groupby("id_student", as_index=False) \
         .agg(failure=("id_assessment", "count"))

3. соединяем - одна колонка с успешно сданным экз, другая с общим количеством. Исп левое соединение,
потому что тех кто сдал успешно меньше, чем тех кто в принципе сдавал. Заполняем 0 пропуски:

In [29]:
all_exams = failure.merge(success_pass, how='left').fillna(0)

4. делим, чтобы получить долю 

In [30]:
frequency = all_exams.assign(frequency=lambda x: x.success / x.failure * 100)[['id_student','frequency']]

>1) по recency (среднее время сдачи одного экзамена) можно получить 2, если значение recency меньше либо равно медиане по recency. В остальных случаях — 1.

>2) по frequency можно получить 1, если значение frequency меньше 50. Можно получить 2, если значение по frequency меньше 100. Можно получить 3 в остальных случаях.

>3) по monetary можно получить 1, если значение monetary меньше 40. Можно получить 2, если значение по monetary меньше либо равно 80. Можно получить 3 в остальных случаях.

In [31]:
RMF = rm.merge(frequency, on='id_student', how='left')

In [32]:
RMF

,id_student,recency,monetary,frequency
0,23698,243.0,80.0,100.0
1,24213,236.0,58.0,100.0
2,27116,243.0,96.0,100.0
3,28046,237.0,40.0,100.0
4,28787,243.0,44.0,100.0
...,...,...,...,...
4628,2694886,236.0,69.0,100.0
4629,2694933,230.0,73.0,100.0
4630,2695608,237.0,73.0,100.0
4631,2697181,230.0,80.0,100.0


In [33]:
RMF["R"] = RMF.recency.apply(lambda x: np.where(x<= RMF.recency.median(), 2, 1)).astype(str)
#приводим к строчному типу, чтобы потом можно было конкатенировать цифры

In [34]:
def freq_score(x):
    if x < 50:
        return 1
    elif 50 <= x < 100:
        return 2
    return 3
RMF['F'] = RMF.frequency.apply(lambda x: freq_score(x)).astype(str)

In [35]:
def mon_score(x):
    if x < 40:
        return 1
    elif 40 <= x <= 80:
        return 2
    return 3
RMF["M"] = RMF.monetary.apply(lambda x: mon_score(x)).astype(str)

In [36]:
RMF = RMF.assign(RFM = lambda x: x.R + x.F + x.M)

In [37]:
RMF.RFM.value_counts()

232    1555
132    1325
133     611
233     578
211     269
111     267
222      18
221       6
122       3
121       1
Name: RFM, dtype: int64